In [1]:
#!pip install dropbox

In [53]:
import json
import dropbox
import pandas as pd
import sys
from collections import Counter
import numpy as np
import pickle
from functools import reduce
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#delet before upload
token = "Insert valid token"

In [3]:
dbx = dropbox.Dropbox(token, timeout=None)

### Downloading & saving entries for dictionaries - Original code

In [81]:
entries = []
lf_res = None
while True:
    if lf_res is None:
        lf_res = dbx.files_list_folder("/whoscored")
    else:
        lf_res = dbx.files_list_folder_continue(lf_res.cursor)
    entries += lf_res.entries
    if not lf_res.has_more:
        break

In [83]:
with open("data_source.txt", "wb") as fp:
    pickle.dump(entries, fp)

### Download & Save Data

In [54]:
with open("data_source.txt", "rb") as fp:
    entries = pickle.load(fp)

In [57]:
def dic_from_entry(en):
    return json.loads(dbx.files_download(en.path_lower)[1].content)

def save_dics(entries):
    bins_start = 34000
    bins_end = 34999
    for bin_num in range(8,10):      
        all_dics = [dic_from_entry(entries[i]) for i in range(bins_start,bins_end)]
        with open("all_dics{}.txt".format(bin_num), "wb") as fp:
            pickle.dump(all_dics, fp)
        print(bins_start, bins_end)
        bins_start+=1000
        bins_end+=1000
        all_dics = []
        print("{} laps are finished".format(bin_num+1))


save_dics(entries)

34000 34999
9 laps are finished
35000 35999
10 laps are finished


### Gen Player DataBase

In [60]:
# Filter out empty dictionaries
def drop_empty(dics):
    dics = [i for i in dics if not (list(i.keys())[0] == "not_found")]
    return dics


def gen_stat(stats, players):
    stat_list = []
    for row in range(len(players)):
        try:
            stat_value = sum(players.iloc[row].stats[stats].values())
            stat_list.append(stat_value)
        except:
            stat_list.append(0)
    return stat_list

# Weighting function for player indicators
def accuracy_weighted(
    indicator1, indicator2, players
): 
    return (
        (players[indicator1] / players[indicator2])
        * ((players[indicator1] / players[indicator1].sum()))
    )

# Match level player Database
def player_match_data(dics):
    players = pd.DataFrame(dics["home"]["players"]).append(
        pd.DataFrame(dics["away"]["players"]), ignore_index=True
    )  # .set_index('playerId')
    events = pd.DataFrame(dics["events"])
    try:
        goal_dict = Counter(list(events.loc[events["isGoal"] == True].playerId))
    except:
        goal_dict = {}


    try:
        card = events.cardType.dropna()
        events["yellow_card"] = events.index.isin(
            card[card.apply(lambda x: x["displayName"] == "Yellow")].index
        )
        events["red_card"] = events.index.isin(
            card[card.apply(lambda x: x["displayName"] == "Red")].index
        )

        players = players.merge(
            events.loc[events["yellow_card"] == True][["playerId", "yellow_card"]],
            how="left",
        )
        players = players.merge(
            events.loc[events["red_card"] == True][["playerId", "red_card"]], how="left"
        )
        players["yellow_card"] = np.where(players["yellow_card"] == True, 1, 0)
        players["red_card"] = np.where(players["red_card"] == True, 1, 0)
    except:
        players["yellow_card"] = 0
        players["red_card"] = 0

    players["goals"] = players["playerId"].map(goal_dict).fillna(0)
    players["isFirstEleven"] = np.where(players["isFirstEleven"] == True, 1, 0)
    players["isManOfTheMatch"] = np.where(players["isManOfTheMatch"] == True, 1, 0)
    players["field"] = np.where(players["field"] == "home", 1, 0)
    players["BMI"] = players["weight"] / (players["height"] ** 2) * 10000

    ratings = []
    for row in range(len(players)):
        try:
            rating = list(players.iloc[row].stats["ratings"].values())[-1]
            ratings.append(rating)
        except:
            ratings.append(np.nan)
    players["ratings"] = ratings

    players["interceptions"] = gen_stat("interceptions", players)

    players["pass_total"] = gen_stat("passesTotal", players)
    players["pass_accurate"] = gen_stat("passesAccurate", players)
    players["pass_rating"] = players["pass_accurate"] / players["pass_total"]
    players["pass_rating_w"] = accuracy_weighted("pass_accurate", "pass_total", players)
    players["pass_rating"] = np.where(players["pass_rating"].isna(), 0, players["pass_rating"])
    players["pass_rating_w"] = np.where(players["pass_rating_w"].isna(), 0, players["pass_rating_w"])

    players["saves"] = gen_stat("totalSaves", players)

    players["dribbles_atempt"] = gen_stat("dribblesAttempted", players)
    players["dribbles_won"] = gen_stat("dribblesWon", players)
    players["dribble_rating"] = players["dribbles_won"] / players["dribbles_atempt"]
    players["dribble_rating_w"] = accuracy_weighted(
        "dribbles_won", "dribbles_atempt", players
    )
    players["dribble_rating"] = np.where(players["dribble_rating"].isna(), 0, players["dribble_rating"])
    players["dribble_rating_w"] = np.where(players["dribble_rating_w"].isna(), 0, players["dribble_rating_w"])
    
    
    players["tackles_total"] = gen_stat("tacklesTotal", players)
    players["tackles_successful"] = gen_stat("tackleSuccessful", players)
    players["tackle_rating"] = players["tackles_successful"] / players["tackles_total"]
    players["tackle_rating_w"] = accuracy_weighted(
        "tackles_successful", "tackles_total", players
    )
    players["tackle_rating"] = np.where(players["tackle_rating"].isna(), 0, players["tackle_rating"])
    players["tackle_rating_w"] = np.where(players["tackle_rating_w"].isna(), 0, players["tackle_rating_w"])
    
    
    players["throwins_total"] = gen_stat("throwInsTotal", players)
    players["throwins_accurate"] = gen_stat("throwInsAccurate", players)
    players["throwin_rating"] = players["throwins_accurate"] / players["throwins_total"]
    players["throwin_rating_w"] = accuracy_weighted(
        "throwins_accurate", "throwins_total", players
    )
    players["throwin_rating"] = np.where(players["throwin_rating"].isna(), 0, players["throwin_rating"])
    players["throwin_rating_w"] = np.where(players["throwin_rating_w"].isna(), 0, players["throwin_rating_w"])
    
    players["goal_keeper"] = np.where(players["position"] == "GK", 1,0)
    
    players["counter"] = 1
    
    try:
        players.drop(
            columns=[
                "stats",
                "subbedOutPeriod",
                "subbedOutExpandedMinute",
                "subbedInPeriod",
                "subbedInExpandedMinute",
                "subbedOutPlayerId",
            ],
            inplace=True,
        )
    except:
        pass

    return players

# Groupby function for full Database
def groupby_function(data_to_filter):
    data_to_filter = data_to_filter.groupby(
        [
            "playerId",
            # "shirtNo", #ettől van sok duplicate
            "name",
            # "position", #meg ettől is
        ],
        as_index=False,
    ).agg(
        {
            "age": "mean",
            "height": "mean",
            "weight": "mean",
            "BMI": "mean",
            "isFirstEleven": "mean",
            "isManOfTheMatch": "mean",
            "field": "mean",
            "subbedInPlayerId": "mean",
            "yellow_card": "mean",
            "red_card": "mean",
            "goals": "mean",
            "ratings": "mean",
            "interceptions": "mean",
            "pass_total": "mean",
            "pass_accurate": "mean",
            "pass_rating": "mean",
            "pass_rating_w": "mean",
            "saves": "mean",
            "dribbles_atempt": "mean",
            "dribbles_won": "mean",
            "dribble_rating": "mean",
            "dribble_rating_w": "mean",
            "tackles_total": "mean",
            "tackles_successful": "mean",
            "tackle_rating": "mean",
            "tackle_rating_w": "mean",
            "throwins_total": "mean",
            "throwins_accurate": "mean",
            "throwin_rating": "mean",
            "throwin_rating_w": "mean",
            "goal_keeper": "mean",
            "counter": "sum",
        }
    )
    return data_to_filter

In [61]:
# Generating full player Database
def player_all():

    dic_num = 5  # number of dictionary list to process- have to set accordingly
    players_data = pd.DataFrame()
    curr_dic = 5

    for data in range(dic_num):
        with open("all_dics{}.txt".format(curr_dic), "rb") as fp: 
            all_dics = list(pickle.load(fp))

        curr_dic += 1
        all_dics = [i for i in all_dics if i is not None]
        all_dics = drop_empty(all_dics)

        matches = map(player_match_data, all_dics)
        player_data_list = list(matches)
        players_all = reduce(
            lambda left, right: left.append(right, ignore_index=True), player_data_list
        )
        players_all = groupby_function(players_all)
        players_data = players_data.append(players_all, ignore_index=True)
        all_dics = []
        print("loops done: {}".format(curr_dic))

    players_data = groupby_function(players_data)
    players_data.drop("subbedInPlayerId", axis = 1, inplace = True)
    players_data["goal_keeper"] = np.where(players_data["goal_keeper"] >= 0.5, 1,0)
    return players_data

In [ ]:
%%time
players_all = player_all()
players_all.to_csv("data_final_5000_2.csv",index=False)

In [ ]:
players_all = pd.read_csv("data_final_5000.csv")

In [ ]:
players_all

In [329]:
players_all[players_all.playerId.duplicated() == True]

,playerId,name,age,height,weight,BMI,isFirstEleven,isManOfTheMatch,field,yellow_card,...,tackles_total,tackles_successful,tackle_rating,tackle_rating_w,throwins_total,throwins_accurate,throwin_rating,throwin_rating_w,goal_keeper,counter
375,2987,Pepe Reina,37.0,188.0,92.0,26.029878,1.000000,0.5,0.500000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1,2
1069,8013,Klaas-Jan Huntelaar,35.0,186.0,83.0,23.991213,0.525000,0.0,0.225000,0.0,...,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,9
1472,10454,Djibril Sidibé,27.0,182.0,71.0,21.434609,0.666667,0.0,0.666667,0.0,...,2.666667,2.0,0.488889,0.045029,8.666667,7.666667,0.60000,0.162944,0,3
2886,23054,Loïc Rémy,33.0,185.0,80.0,23.374726,0.500000,0.0,1.000000,0.0,...,1.000000,0.5,0.250000,0.007812,0.000000,0.000000,0.00000,0.000000,0,2
3781,31826,Séamus Coleman,31.0,177.0,67.0,21.385936,1.000000,0.0,1.000000,0.0,...,4.000000,3.0,0.750000,0.060527,14.000000,10.500000,0.78655,0.230518,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16447,360682,Andrey Kozlov,31.0,186.0,88.0,25.436467,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,2
16724,362815,Viktor Gyökeres,21.0,187.0,86.0,24.593211,0.000000,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0,1
16740,363083,Javairô Dilrosun,21.0,174.0,74.0,24.441802,1.000000,0.0,1.000000,0.0,...,1.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,0,1
16946,366705,Giorgos Masouras,26.0,184.0,76.0,22.448015,1.000000,0.0,1.000000,0.0,...,3.000000,3.0,1.000000,0.090909,0.000000,0.000000,0.00000,0.000000,0,1
